In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np

In [11]:
df = pd.read_table('../Data/shopping.txt')
df

,5,배공빠르고 굿
0,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
1,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
2,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
3,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
4,2,비추합니다 계란 뒤집을 때 완전 불편해요 ㅠㅠ 코팅도 묻어나고 보기엔 예쁘고 실용적...
...,...,...
199994,2,장마라그런가!!! 달지않아요
199995,5,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199996,5,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199997,5,넘이쁘고 쎄련되보이네요~


In [12]:
df.columns = ['label', 'text']

In [13]:
df['label'].replace([(1, 2), (3, 4, 5)], [0, 1], inplace=True)

In [14]:
df['label'].value_counts()

label
0    100037
1     99962
Name: count, dtype: int64

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199999 entries, 0 to 199998
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   label   199999 non-null  int64 
 1   text    199999 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.1+ MB


In [17]:
df['text'].nunique()

199907

In [19]:
df.drop_duplicates(subset=['text'], inplace=True)

In [20]:
ydata=df['label']
xdata=df['text']

In [48]:
from sklearn.model_selection import train_test_split

#층화추출:종류에따라 균등하게 분리하여 추출
xtrain, xtest, ytrain, ytest=train_test_split(xdata,ydata,test_size=0.2,
                                              random_state=20230728,
                                              stratify=ydata)

In [27]:
f = open('../Data/korean_stopwords.txt')
text = f.readlines()

In [28]:
text

['아\n',
 '휴\n',
 '아이구\n',
 '아이쿠\n',
 '아이고\n',
 '어\n',
 '나\n',
 '우리\n',
 '저희\n',
 '따라\n',
 '의해\n',
 '을\n',
 '를\n',
 '에\n',
 '의\n',
 '가\n',
 '으로\n',
 '로\n',
 '에게\n',
 '뿐이다\n',
 '의거하여\n',
 '근거하여\n',
 '입각하여\n',
 '기준으로\n',
 '예하면\n',
 '예를 들면\n',
 '예를 들자면\n',
 '저\n',
 '소인\n',
 '소생\n',
 '저희\n',
 '지말고\n',
 '하지마\n',
 '하지마라\n',
 '다른\n',
 '물론\n',
 '또한\n',
 '그리고\n',
 '비길수 없다\n',
 '해서는 안된다\n',
 '뿐만 아니라\n',
 '만이 아니다\n',
 '만은 아니다\n',
 '막론하고\n',
 '관계없이\n',
 '그치지 않다\n',
 '그러나\n',
 '그런데\n',
 '하지만\n',
 '든간에\n',
 '논하지 않다\n',
 '따지지 않다\n',
 '설사\n',
 '비록\n',
 '더라도\n',
 '아니면\n',
 '만 못하다\n',
 '하는 편이 낫다\n',
 '불문하고\n',
 '향하여\n',
 '향해서\n',
 '향하다\n',
 '쪽으로\n',
 '틈타\n',
 '이용하여\n',
 '타다\n',
 '오르다\n',
 '제외하고\n',
 '이 외에\n',
 '이 밖에\n',
 '하여야\n',
 '비로소\n',
 '한다면 몰라도\n',
 '외에도\n',
 '이곳\n',
 '여기\n',
 '부터\n',
 '기점으로\n',
 '따라서\n',
 '할 생각이다\n',
 '하려고하다\n',
 '이리하여\n',
 '그리하여\n',
 '그렇게 함으로써\n',
 '하지만\n',
 '일때\n',
 '할때\n',
 '앞에서\n',
 '중에서\n',
 '보는데서\n',
 '으로써\n',
 '로써\n',
 '까지\n',
 '해야한다\n',
 '일것이다\n',
 '반드시\n',
 '할줄알다\n',
 '할수있

In [34]:
xtrain[1]

'아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요.'

In [35]:
from string import punctuation

In [49]:
from konlpy.tag import Okt

okt = Okt()
x_train=[]
for sent in xtrain:
  tok_sent=okt.morphs(str(sent)) #토큰화
  sw_rem_sent=[w for w in tok_sent if not w in text if w not in punctuation]
  x_train.append(sw_rem_sent)

In [50]:
x_test=[]
for sent in xtest:
  tok_sent=okt.morphs(str(sent)) #토큰화
  sw_rem_sent=[w for w in tok_sent if not w in text if w not in punctuation]
  x_test.append(sw_rem_sent)

In [58]:
temp = []
for i in x_train:
    i = ' '.join(i)
    temp.append(i)

In [59]:
temp

['그림 도 아니고 사진 같은데 비싼 듯 요',
 '손잡이 부분 이 한번 쓰니 떨어지고 드라이 할 때 머리 에 촥촥 감기 는 느낌 이 좀 부족하네요 ㅣㅣ',
 '빠른 배송 과 넉넉한 양 이 좋네요 .. 유통 기한 은 얼마 안남았지만 잘 먹이겟습니다',
 '칼 배송 이라고 해서 급히 주문 했는데 배송 까지 4일 걸려서 결국 여행 갈 때 못 들 고 갔네요 ..',
 '넘 맛있어요 ㅎㅎ 다 먹고 또 주문 해야지',
 '싸고 좋습니다 앞 으로도 계속 사려구요',
 '생각 보다 편하지 않아요',
 '맛 도 좋고 양도 넉넉해서 다시 재 구매 의향 이 있습니다 .~~^^',
 '감사합니다 잘 받았고 응대 도 너무 좋았습니다',
 '식당 비치 용 으로 구매 했어요 다 써서 재구매 하려구요 ㅎㅎ',
 '다 부서진거만 왔어요',
 '추천 합니다 설치을 잘못 해서 톡 으로 전화 로 도움 받았어요 친절하게 잘 설명 해주시 고 사진 으로 설치 과정 이 쉽게 돼 있더라구요',
 '처음 에 받고 제대로 된것도 안오더니 개별 로 사지마 셈',
 '좋아요 아기 때메 폰 을 ㅈ 두고나 두는 일이 많았는데 줄이오고 난후 로는 매일 들 고 다니네요 ~~ 시 엄닠 너 두 샀어요',
 '아쓉 쪽발이 직 배송 인줄 알았으면 여기 서안 샀는데',
 '생각 보다 무거워 서 깜짝 놀랐어요 크기 도크 고요 ㅎ 급하게 사느라 사이 즈 안보 고 후기 만 보고 샀는데 높이 도 높고 사진 보다 가로 길이 도 기네 요 ㅎ 군더더기 없어요 가격 은 좀비 싸지만 ㅜ',
 '짱짱맨 인 듯 ㅎㅎ',
 '너무 비싼거 같다 조금 저렴하면 좋을 듯',
 '조립 하다가 아래층 사람 올라와서 한소리 들었습니다 사람 없는 시간대 에 조립 하세요',
 '별반 다르지 않는 평범한 설탕 이고 주 문 마감 이 오전 9시 라 3일 만에 받았네요',
 '생각 보다 그저 그래요',
 '완전 부드럽고 맛있어요',
 '손잡이 가 잘 뜯어져서 아쉬워요',
 '가성 비 좋고 편해요 부모님 사 드렸는데 좋다고 하십니다',
 '선물 한 거 라서 맛 이 어떤

In [60]:
x_train = temp

In [61]:
x_train

['그림 도 아니고 사진 같은데 비싼 듯 요',
 '손잡이 부분 이 한번 쓰니 떨어지고 드라이 할 때 머리 에 촥촥 감기 는 느낌 이 좀 부족하네요 ㅣㅣ',
 '빠른 배송 과 넉넉한 양 이 좋네요 .. 유통 기한 은 얼마 안남았지만 잘 먹이겟습니다',
 '칼 배송 이라고 해서 급히 주문 했는데 배송 까지 4일 걸려서 결국 여행 갈 때 못 들 고 갔네요 ..',
 '넘 맛있어요 ㅎㅎ 다 먹고 또 주문 해야지',
 '싸고 좋습니다 앞 으로도 계속 사려구요',
 '생각 보다 편하지 않아요',
 '맛 도 좋고 양도 넉넉해서 다시 재 구매 의향 이 있습니다 .~~^^',
 '감사합니다 잘 받았고 응대 도 너무 좋았습니다',
 '식당 비치 용 으로 구매 했어요 다 써서 재구매 하려구요 ㅎㅎ',
 '다 부서진거만 왔어요',
 '추천 합니다 설치을 잘못 해서 톡 으로 전화 로 도움 받았어요 친절하게 잘 설명 해주시 고 사진 으로 설치 과정 이 쉽게 돼 있더라구요',
 '처음 에 받고 제대로 된것도 안오더니 개별 로 사지마 셈',
 '좋아요 아기 때메 폰 을 ㅈ 두고나 두는 일이 많았는데 줄이오고 난후 로는 매일 들 고 다니네요 ~~ 시 엄닠 너 두 샀어요',
 '아쓉 쪽발이 직 배송 인줄 알았으면 여기 서안 샀는데',
 '생각 보다 무거워 서 깜짝 놀랐어요 크기 도크 고요 ㅎ 급하게 사느라 사이 즈 안보 고 후기 만 보고 샀는데 높이 도 높고 사진 보다 가로 길이 도 기네 요 ㅎ 군더더기 없어요 가격 은 좀비 싸지만 ㅜ',
 '짱짱맨 인 듯 ㅎㅎ',
 '너무 비싼거 같다 조금 저렴하면 좋을 듯',
 '조립 하다가 아래층 사람 올라와서 한소리 들었습니다 사람 없는 시간대 에 조립 하세요',
 '별반 다르지 않는 평범한 설탕 이고 주 문 마감 이 오전 9시 라 3일 만에 받았네요',
 '생각 보다 그저 그래요',
 '완전 부드럽고 맛있어요',
 '손잡이 가 잘 뜯어져서 아쉬워요',
 '가성 비 좋고 편해요 부모님 사 드렸는데 좋다고 하십니다',
 '선물 한 거 라서 맛 이 어떤

In [62]:
x_test

[['기',
  '기대이하네요',
  '아들',
  '방이',
  '넘',
  '더워서',
  '선풍기',
  '와',
  '겸',
  '해서',
  '사용',
  '하려고',
  '삿는데',
  '차라리',
  '선풍기',
  '한',
  '대',
  '를더',
  '사는것이',
  '훨씬',
  '좋을',
  '뜻'],
 ['재구매',
  '가격',
  '이',
  '좋아서',
  '2',
  '번째',
  '구매',
  '인데',
  '앞',
  '으로',
  '자주',
  '구매',
  '할것',
  '같습니다'],
 ['완전', '싸구려', '옷', '같아요', 'ㅠㅠ'],
 ['리뷰',
  '보고',
  '괜찮다고',
  '해서',
  '여러',
  '개',
  '주문',
  '했는데요',
  '하나',
  '요리',
  '해서',
  '먹는데',
  '한',
  '입',
  '먹고',
  '버렸어요',
  '저',
  '한테',
  '배송',
  '온것만',
  '그런',
  '건',
  '지',
  '냄새',
  '가',
  '너무',
  '너무',
  '심했어요',
  ',,,,',
  '생각',
  '만해',
  '도',
  '울렁거러',
  '요'],
 ['재구매',
  '고소하고',
  '맛있어요',
  '임신',
  '전',
  '부터',
  '계속',
  '먹고',
  '있어요',
  '단맛',
  '이',
  '없어서',
  '콩국수',
  '맛',
  '이라고',
  '생각',
  '하면되',
  '고',
  '고소해서',
  '잘',
  '먹어요'],
 ['배송비',
  '를',
  '5000원',
  '이나',
  '받네요',
  '추가',
  '로',
  '2000원',
  '더',
  '보내라고',
  '해서',
  '보냈더니',
  '눌건',
  '받아',
  '보니',
  '아이스팩',
  '작은거',
  '하나',
  '넣어',
  '보냈네요',
  '..',
  '인터넷',
  '거래',
  '많이',
  '해봤지만'

In [64]:
x_train = np.array(x_train).reshape(-1, 1)

In [65]:
x_train

array([['그림 도 아니고 사진 같은데 비싼 듯 요'],
       ['손잡이 부분 이 한번 쓰니 떨어지고 드라이 할 때 머리 에 촥촥 감기 는 느낌 이 좀 부족하네요 ㅣㅣ'],
       ['빠른 배송 과 넉넉한 양 이 좋네요 .. 유통 기한 은 얼마 안남았지만 잘 먹이겟습니다'],
       ...,
       ['가격 대비 만족 입니다 이전 에 비싼것보다 먼지 가 조금 더 많이 끼는것 빼구요'],
       ['오 ~~~~ 뽑기 잘 한거 같아요 가죽 의자 들 처럼 많이 편하진 않지만 불편한거 없어요 ㅋㅋㅋ 게이밍 이랑 고민 했는데 너무 잘 샀네요 한 달 써 보고 후기 또 남기러 올게요'],
       ['가성 비 최고 입니다 색상 별로 3 개 씩 구 매 햇 는데 너무 좋아요']], dtype='<U189')

In [63]:
temp = []
for i in x_test:
    i = ' '.join(i)
    temp.append(i)

x_test = temp
x_test

['기 기대이하네요 아들 방이 넘 더워서 선풍기 와 겸 해서 사용 하려고 삿는데 차라리 선풍기 한 대 를더 사는것이 훨씬 좋을 뜻',
 '재구매 가격 이 좋아서 2 번째 구매 인데 앞 으로 자주 구매 할것 같습니다',
 '완전 싸구려 옷 같아요 ㅠㅠ',
 '리뷰 보고 괜찮다고 해서 여러 개 주문 했는데요 하나 요리 해서 먹는데 한 입 먹고 버렸어요 저 한테 배송 온것만 그런 건 지 냄새 가 너무 너무 심했어요 ,,,, 생각 만해 도 울렁거러 요',
 '재구매 고소하고 맛있어요 임신 전 부터 계속 먹고 있어요 단맛 이 없어서 콩국수 맛 이라고 생각 하면되 고 고소해서 잘 먹어요',
 '배송비 를 5000원 이나 받네요 추가 로 2000원 더 보내라고 해서 보냈더니 눌건 받아 보니 아이스팩 작은거 하나 넣어 보냈네요 .. 인터넷 거래 많이 해봤지만 일반 배송 에 택배 비 5000원 은 처음 이네 요 너무 불쾌합니다',
 '글리터 가 안 왔어요 ㅠㅠ 다른건 왓는데',
 '배송 은 2일 만에 왔구요 잘쓰겠습니다',
 '재구매 짜서 쓰는 곰 파이 제 거제 는 힘 을 꽉 줘 야해요 ㅠ',
 '항상 쓰는 제품 으로 만족합니다',
 '간편한건 좋지만 가공 식품 먹는 맛 이라 실망 이 요',
 '재질 이나 퀄리티 는 좋은 것 같은데 오토바이 곡 이 많은 곳 을 할려니 초보자 는 힘드 네 유 ㅠㅠ 제품 은 만족합니다',
 '사만 결재 하고 ... 무슨 상담 을 햇으 며 무슨 조언 을 받앗는가 싶습니다',
 '씌우기 간단한데 고정 스티로폼 이 자꾸 빠지네요 ㅠ',
 '너무 커서 쓰기 힘들 정도 네 요 .. ㅋㅋ',
 '재구매 좋아요 잘쓰고있 습 니당 ㅎㅎ',
 '안 닦이내요 기대 많이 하고 구 매 한 제품 인데',
 '받아서 씌워 보니 좀 크고 화면 에서 보는것 보다는 예쁘지가 않아요 ... ㅠㅠ',
 '좋네요 저렴하고 아이 가 좋아합니다',
 '좋아요 아주 가볍고 착용 안 한것 같은 느낌 이 들 정도 로 편안해요 가볍게 착용 하기에 좋을것 같아요 추천 합니다',
 '물 빠짐이 

In [66]:
x_test = np.array(x_test).reshape(-1, 1)
x_test

array([['기 기대이하네요 아들 방이 넘 더워서 선풍기 와 겸 해서 사용 하려고 삿는데 차라리 선풍기 한 대 를더 사는것이 훨씬 좋을 뜻'],
       ['재구매 가격 이 좋아서 2 번째 구매 인데 앞 으로 자주 구매 할것 같습니다'],
       ['완전 싸구려 옷 같아요 ㅠㅠ'],
       ...,
       ['가격 두 저렴하고 깔끔하네요 잘쓸께요'],
       ['포장 도 안되서 두 개 가 엄청 굴러다니다 박스 도 찢어져 왔더라고요 포장 에 조금 신 경 쓰시 고 냄새 만 쫌 들 났다면 제품 은 별 다섯 개 짜 리 예요 효과 는 짱 입니다'],
       ['어지러워 죽을거 같 애 요 ㅕ 싼마이 로 씁니댜']], dtype='<U188')

In [68]:
tok = Tokenizer()
tok.fit_on_texts(xdata) #xtrain를 코퍼스로하여 fitting(단어별로 index가 부여됨)
xtrain_enc = tok.texts_to_sequences(xtrain)
xtest_enc = tok.texts_to_sequences(xtest)

In [69]:
xtrain_enc

[[5598, 179, 306274, 88534],
 [567, 135, 39571, 681, 38179, 2304, 45170, 6919, 221, 6, 275403],
 [1567, 2939, 302, 13, 2868, 304906, 83230],
 [155084, 68, 4849, 44, 4504, 2808, 1197, 303, 3116, 69511],
 [324131, 24, 46145],
 [36947, 994, 48, 2812],
 [10, 35594],
 [108, 35, 369, 1767, 37, 2320, 83, 31403, 54],
 [26, 2, 2759, 4440, 1, 268],
 [139435, 84, 7, 1330, 139436],
 [7, 191938, 63],
 [255,
  253954,
  3388,
  13029,
  3855,
  5703,
  342,
  856,
  253955,
  2141,
  527,
  27040,
  253956,
  10520,
  3034],
 [509, 462, 131, 31296, 196456, 9184, 10855],
 [3, 293019, 293020, 3598, 293021, 293022, 11600, 293023],
 [110701, 110702, 4401, 2149, 110703],
 [10, 1226, 4971, 350198, 350199, 350200, 1088, 350201, 350202, 350203],
 [247341, 30],
 [1, 1875, 8542, 28, 8382, 629, 351],
 [5973, 201618, 2111, 2731, 35438, 2934, 201619, 201620, 36760],
 [5658, 11185, 878, 6430, 131717, 131718, 131719, 3587, 915],
 [10, 1065],
 [275456],
 [1075, 2, 2348, 278],
 [74, 35, 139, 1026, 1103, 318, 5337],


In [70]:
xtest_enc

[[9342,
  9390,
  247227,
  25,
  1946,
  83674,
  36687,
  711,
  736,
  523,
  3444,
  247228,
  38279,
  247229],
 [4, 88, 81, 1581, 1369, 405, 187, 18385, 120],
 [33, 96304, 16],
 [1217,
  37284,
  723,
  6588,
  112,
  18250,
  870,
  9448,
  368,
  2480,
  14,
  53286,
  933,
  53,
  252,
  10969,
  64632,
  322841],
 [4, 1302, 43, 84058, 48, 125, 34, 343301, 30862, 17725, 343302, 12719, 621],
 [3699,
  329655,
  9208,
  804,
  13199,
  9,
  17414,
  68,
  19111,
  329656,
  329657,
  1403,
  112,
  329658,
  1344,
  16111,
  8,
  10601,
  354,
  726,
  2037,
  81996,
  1634,
  1,
  8098],
 [64777, 131387, 810, 6960],
 [52, 3950, 2853, 1363, 557],
 [4, 352884, 352885, 2923, 352886],
 [57, 395, 1152, 18],
 [130723, 1151, 130724, 8480, 17471],
 [7496, 2854, 766, 336, 19878, 47492, 144545, 37085, 20326, 144546, 114, 18],
 [290689, 343, 17877, 290690, 343, 85106, 290691, 1748],
 [32191, 22582, 2094, 11328, 141, 1382, 27],
 [1, 194, 287, 8225, 3965, 86],
 [4, 3, 331141],
 [359053, 487

In [71]:
maxLen=max(len(s) for s in xtrain_enc)
maxLen

47

In [72]:
xtrain_padded=pad_sequences(xtrain_enc, maxlen=maxLen)

In [73]:
xtrain_padded.shape

(159925, 47)

In [74]:
xtrain_padded[0]

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,   5598,    179, 306274,  88534])

In [75]:
w2i=tok.word_index
vocab_size=len(w2i)+1 #1번부터 단어의 번호 부여, 0 패딩을 했으므로 1을 더하여 단어사전크기를 설정
vocab_size

364926

In [76]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential

In [77]:
model=Sequential()
model.add(Embedding(vocab_size, 32))
model.add(SimpleRNN(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [78]:
history=model.fit(xtrain_padded, ytrain, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
3999/3999 [==============================] - 370s 92ms/step - loss: 0.3706 - accuracy: 0.8370 - val_loss: 0.3148 - val_accuracy: 0.8679
Epoch 2/5
3999/3999 [==============================] - 372s 93ms/step - loss: 0.2835 - accuracy: 0.8889 - val_loss: 0.3097 - val_accuracy: 0.8710
Epoch 3/5
3999/3999 [==============================] - 370s 93ms/step - loss: 0.2327 - accuracy: 0.9161 - val_loss: 0.3194 - val_accuracy: 0.8724
Epoch 4/5
3999/3999 [==============================] - 390s 97ms/step - loss: 0.1770 - accuracy: 0.9412 - val_loss: 0.3843 - val_accuracy: 0.8396
Epoch 5/5
3999/3999 [==============================] - 425s 106ms/step - loss: 0.1416 - accuracy: 0.9549 - val_loss: 0.3320 - val_accuracy: 0.8750
